<center><h2>Gen AI: Langchain and Prompting Essentials</h2></center

#### 4.1: Elements of good prompt
- Clarity and Specificity
- Output Format
- Context Background
- Tone, Style & Constraint

#### 4.2: Zero-shot, One-shot, Few-shot Prompting
- Zero-shot, we provide only prompt.
- In One-shot, we provide 1 example along with prompt.
- In Few-shot prompting, we provided few example along with prompt.

#### 4.3 & 4.4: installation of Langchain, Ollama, Groq
- Ollama allows you to run the LLM models locally.
- Groq allows to call LLMs in the cloud without any charges.
- The inference (time it takes LLM to generate response) in Groq is very fast.

#### 4.5: Calling LLM from Langchain

In [3]:
# Langchain package installation guide: https://python.langchain.com/docs/how_to/installation/

##### Load API Key

In [2]:
import os
os.environ["GROQ_API_KEY"] = "Enter the api key"

##### Calling LLM in Cloud using Groq

In [3]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile"
)
llm.invoke("The person to go to moon was")

AIMessage(content='The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 42, 'total_tokens': 117, 'completion_time': 0.097952506, 'prompt_time': 0.01160704, 'queue_time': 0.080937394, 'total_time': 0.109559546}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd470ff6-c147-482a-8835-68c32c8b0ccf-0', usage_metadata={'input_tokens': 42, 'output_tokens': 75, 'total_tokens': 117})

In [6]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.6,
    max_tokens=15
)
llm.invoke("The person to go to moon was")

AIMessage(content='The first person to walk on the moon was Neil Armstrong. He stepped out', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 42, 'total_tokens': 57, 'completion_time': 0.007680503, 'prompt_time': 0.031898605, 'queue_time': 0.0496153, 'total_time': 0.039579108}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'length', 'logprobs': None}, id='run--ddfe0391-94e2-4c11-806b-2f0d9f47ee24-0', usage_metadata={'input_tokens': 42, 'output_tokens': 15, 'total_tokens': 57})

##### Local Call to LLM Using- Ollama

In [7]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b",temperature=0, num_predict=15)
llm.invoke("The person to go to moon was")

AIMessage(content='Neil Armstrong and Edwin "Buzz" Aldrin were the first humans to walk', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-08-05T19:18:53.7989118Z', 'done': True, 'done_reason': 'length', 'total_duration': 3962779300, 'load_duration': 2715785800, 'prompt_eval_count': 32, 'prompt_eval_duration': 323018800, 'eval_count': 15, 'eval_duration': 922919700, 'model_name': 'llama3.2:1b'}, id='run--71f03823-32c9-4bfc-855f-c6a6faab8a30-0', usage_metadata={'input_tokens': 32, 'output_tokens': 15, 'total_tokens': 47})

#### 4.6: PromptTemplate and Chain

In [8]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

In [9]:
prompt = PromptTemplate.from_template("Write two line poem on {topic}")
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Write two line poem on {topic}')

In [10]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile")

chain = prompt | llm

response = chain.invoke({'topic': 'butterfly'})
response

AIMessage(content='A delicate butterfly dances free,\nSpreading colorful wings, a wonder to see.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 41, 'total_tokens': 58, 'completion_time': 0.048548222, 'prompt_time': 0.011238493, 'queue_time': 0.045031857, 'total_time': 0.059786715}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6399de1f-4a8d-4104-ba1b-1b0640e0b693-0', usage_metadata={'input_tokens': 41, 'output_tokens': 17, 'total_tokens': 58})

In [11]:
print(response.content)

A delicate butterfly dances free,
Spreading colorful wings, a wonder to see.


#### 4.7: Output Parser

In [12]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException

In [13]:
prompt = '''
From the below news article, extract revenue and eps in JSON format containing the
following keys: 'revenue_actual', 'revenue_expected', 'eps_actual', 'eps_expected'. 

Each value should have a unit such as million or billion.

Only return the valid JSON. No preamble.

Article
=======
{article}
'''

pt = PromptTemplate.from_template(prompt)
pt

PromptTemplate(input_variables=['article'], input_types={}, partial_variables={}, template="\nFrom the below news article, extract revenue and eps in JSON format containing the\nfollowing keys: 'revenue_actual', 'revenue_expected', 'eps_actual', 'eps_expected'. \n\nEach value should have a unit such as million or billion.\n\nOnly return the valid JSON. No preamble.\n\nArticle\n=======\n{article}\n")

In [14]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
chain = pt | llm

In [15]:
article_text = '''
Here’s what the company reported compared with what Wall Street was expecting, based on a survey of analysts by LSEG:

Earnings per share: 72 cents, adjusted vs. 58 cents expected
Revenue: $25.18 billion vs. $25.37 billion expected
'''
response = chain.invoke({'article': article_text})
response.content

'{"revenue_actual": "25.18 billion", "revenue_expected": "25.37 billion", "eps_actual": "72 cents", "eps_expected": "58 cents"}'

In [16]:
article_text = '''
Here’s how the iPhone maker did versus LSEG consensus estimates for the quarter ending Sept. 28:  

Earnings per share: $1.64, adjusted, versus $1.60 estimated 
Revenue: $94.93 billion vs. $94.58 billion estimated 
'''
response = chain.invoke({'article': article_text})
response.content

'{"revenue_actual": "94.93 billion", "revenue_expected": "94.58 billion", "eps_actual": "1.64", "eps_expected": "1.60"}'

In [17]:
parser = JsonOutputParser()
res = parser.parse(response.content)
res

{'revenue_actual': '94.93 billion',
 'revenue_expected': '94.58 billion',
 'eps_actual': '1.64',
 'eps_expected': '1.60'}

In [18]:
res['revenue_actual']

'94.93 billion'